In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import sys
import os

if os.access('Classification.xlsx', os.F_OK): 
    classi = pd.read_excel('Classification.xlsx', 'Region', index_col=None, na_values=['NA'])
    lat1 = classi['Upper-Left Lat.']
    lat2 = classi['Lower-Right Lat.']
    logi1 = classi['Upper-Left Longi.']
    logi2 = classi['Lower-Right Longi.']
    Num = classi['Number']
else: 
    print('Error: Can not find Classification.xlsx - Sheet\'Region\', the function of Classification is invalid.')

def numberjudge(x):
    for charc in str(x):
        L= [x for x in range(10)]
        L= list(map(str, L))
        if charc in L:
            pass
        else:
            return False
    return True
def zero(x):
    x = str(x)
    if len (x) == 1:
        x = '0' + x
    return x
def classify(lat, logi):
    for z in range(0,len(classi)):   
        if lat2[z]<lat<lat1[z] and logi1[z]<logi<logi2[z]:
            return Num[z]
        
frq, prd, fstartdate, pref, finalrow = 'M', 12, '1998/03', 'GIS_traj_ps_', 1
if os.access('CONFIG', os.F_OK): 
    loadpara = []
    loadpara = pd.read_csv('CONFIG', header=None, sep=",")
    frq = str(loadpara[0][1])
    prd = int(loadpara[0][2])
    fstartdate = str(loadpara[0][3])
    pref = str(loadpara[0][4])
    finalrow = int(loadpara[0][5])
    print ('Loaded the parameters: frq=%s, prd=%s, fstartdate=%s, pref=%s, finalrow=%s' % (frq, prd, fstartdate, pref, finalrow))
else:
    print ('Reset the parameters: frq=%s, prd=%s, fstartdate=%s, pref=%s, finalrow=%s' % (frq, prd, fstartdate, pref, finalrow))
    
while True:
    runornot = input('Please input the command:\'end\', \'parascan\', \'save\', or reset the parameter \'frq\', \'prd\', \'fstartdate\', \'pref\', \'finalrow\',\nafter reset the parameters, for running the programme, please input \'classify\', \'merge\', : ')
    if runornot == 'end':
        break
    elif runornot == 'parascan':
        print ('Now, the parameters: frq=%s, prd=%s, fstartdate=%s, pref=%s, finalrow=%s.' % (frq, prd, fstartdate, pref, finalrow))
        continue
    elif runornot == 'save':
        savepara = [frq, prd, fstartdate, pref, finalrow]
        saveparadata = pd.DataFrame(savepara)
        saveparadata.to_csv('CONFIG', sep=',', index=False)
        print ('Parameters save as: frq=%s, prd=%s, fstartdate=%s, pref=%s, finalrow=%s' % (frq, prd, fstartdate, pref, finalrow))
        continue
    elif runornot == 'frq':
        frq = input('Please input the frequence,D or M or Y.')
        if frq == 'D' or 'M' or 'Y':
            print ('Set frq as \'%s\'' % (frq))
            pass
        else:
            print ('The input is wrong, reset frq as \'M\'')
            frq = 'M'
        continue
    elif runornot == 'prd':
        prd = input('Please input the period, it should be a integer.')
        if numberjudge(prd) == True:
            prd = int (prd)
            print ('Set prd as \'%s\'' % (prd))
            pass
        else:
            while numberjudge(prd) != True:
                prd = input('Prd should be a integer. Please reset it again.')
    elif runornot == 'fstartdate':
        fstartdate = input('Please input the start date of file, it should be corresponded with frequece, like: input 1998/01/01 for frequence \'D\'.')
        print ('Set fstartdate as \'%s\'' % (fstartdate))
        if len (fstartdate) == 4 and frq != 'Y':
            print ('The start date of file is not corresponded with frequece, the frequce reset as Y')
            frq = 'Y'
        elif len (fstartdate) == 7 and frq != 'M':
            print ('The start date of file is not corresponded with frequece, the frequce reset as M')
            frq = 'M'
        elif len (fstartdate) == 10 and frq != 'D':
            print ('The start date of file is not corresponded with frequece, the frequce reset as D')
            frq = 'D'
        continue
    elif runornot == 'pref':
        pref = input('Please input the prefixion of the trajectories, the default of HYSPLIT is \'GIS_traj_ps_\'')
        pref = str(pref)
        continue
    elif runornot == 'finalrow':
        finalrow = input('Please input the row for classification, default is 1, it means the 1st row which is counted backwards.')
        if numberjudge(finalrow) == True:
            finalrow = int (finalrow)
            print ('Set finalrow as \'%s\'' % (finalrow))
            pass
        else:
            while numberjudge(finalrow) != True:
                finalrow = input('Variable \'finalrow\' should be a integer. Please reset it again.')
    elif runornot == 'merge' or runornot == 'classify':
        dates = pd.date_range(fstartdate, periods = prd,freq = frq)
        filesdate = []
        for x in range (0, prd):
            if frq == 'D':
                filesdate.append (str(dates[x])[0:4] + str(dates[x])[5:7] + str(dates[x])[8:10])
            elif frq == 'M':
                filesdate.append (str(dates[x])[0:4] + str(dates[x])[5:7])
            elif frq == 'Y':
                filesdate.append (str(dates[x])[0:4])
        R = [y for y in range(1,100)]
        LR = list(map(zero, R))
        compname = []
        for file in filesdate:
                for trajnumber in LR:
                    compname.append (file + '\\' + pref + trajnumber + '.txt')    
        print ('The potential path of trajectories successfully generated: ')
        print (compname[:3])
        print ('...')
        print (compname[-3:])
        
        if runornot == 'classify':
            progressrate= 0
            showprogressrate= 0
            Region = []
            Filename = []
            Latitude_classfication = []
            Logitude_classfication = []
            rowforjudge = []
            for xx in compname:
                progressrate = (progressrate + (1 / len(compname)))
                showprogressrate = int(progressrate * 100) 
                print("\r", end="")
                print("Classify progress: {}%: ".format(showprogressrate), "▋" * (showprogressrate // 5), end="")
                sys.stdout.flush()
                if os.access(xx, os.F_OK): 
                    hys = []
                    hys = pd.read_csv(xx, header=None, sep=",")
                    lenth = len(hys)-1
                    hys= hys[0:lenth]
                    last = len (hys)
                    lat = float(hys.loc[last - finalrow,[2]])
                    logi = float(hys.loc[last - finalrow,[1]])
                    Latitude_classfication.append (lat)
                    Logitude_classfication.append (logi)
                    rowforjudge.append (finalrow)
                    Region.append (classify(lat, logi))
                    Filename.append (xx)
                    #break
            Regiondata = pd.DataFrame({'FileName': Filename,
                                       'Final Row': rowforjudge,
                                       'Latitude': Latitude_classfication,
                                       'Longitude': Logitude_classfication,
                                       'Region': Region})
            TIMENOW =pd.datetime.now()
            OUTPUTTIME = TIMENOW.strftime('%d%H%M%S')
            Regiondata.to_excel('Classi_result_'+ OUTPUTTIME + '.xlsx', sheet_name='Classification Result')
            print ('')
            print ('Classification is successfully done, it output as: ' + 'Classi_result_'+ OUTPUTTIME + '.xlsx\n' )
            
        if runornot == 'merge':
            progressrate= 0
            showprogressrate= 0
            trajectorycount = -1
            countfm = []
            latifm = []
            logifm = []
            Filenamefm = []
            for xx in compname:
                progressrate = (progressrate + (1 / len(compname)))
                showprogressrate = int(progressrate * 100) 
                print("\r", end="")
                print("Merge progress: {}%: ".format(showprogressrate), "▋" * (showprogressrate // 5), end="")
                sys.stdout.flush()
                if os.access(xx, os.F_OK): 
                    hysfm = []
                    hysfm = pd.read_csv(xx, header=None, sep=",")
                    lenthfm = len(hysfm)-1
                    hysfm = hysfm[0:lenthfm]
                    trajectorycount = trajectorycount + 1
                    for yy in range(0,lenthfm):
                        latifmt = float(hysfm.loc[yy,[2]])
                        logifmt = float(hysfm.loc[yy,[1]])
                        Filenamefm.append (xx)
                        countfm.append (trajectorycount)
                        latifm.append (latifmt)
                        logifm.append (logifmt)
            Mergedata = pd.DataFrame({'FileName': Filenamefm,
                                       'Line': countfm,
                                       'Latitude': latifm,
                                       'Longitude': logifm,})
            TIMENOW =pd.datetime.now()
            OUTPUTTIME = TIMENOW.strftime('%d%H%M%S')
            Mergedata.to_excel('Merge_result_'+ OUTPUTTIME + '.xlsx', sheet_name='Merge Result')
            print ('')
            print ('Merge is successfully done, it output as: ' + 'Merge_result_'+ OUTPUTTIME + '.xlsx\n' )
        continue